In [1]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"

In [2]:
# number_of_shots = 3

In [3]:
nbit = 4
llamasize = '7B'
# llamasize = '13B'
temp = 0.1

In [4]:
experiment_name = "2023.12.14-02_LLama2_7B-chat-hf_4-bit_float-16_zero-shot_FINETUNED"

In [5]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"

# top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_but_with_different_classes_with_2023.12.11_input_data.pickle"

output_data_path = base_path + "/../Llama-2_FineTuning/data/output/" + experiment_name
log_filename = base_path + "/../Llama-2_FineTuning/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


# LOGIN DA HUGGING FACE

In [ ]:
# @title
# import os
# from IPython.display import clear_output
# os.environ["OPENAI_API_KEY"] = input("INSERT YOU OPENAI API KEY:")
# clear_output()

from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [ ]:
# @title
!huggingface-cli login --token $huggingface_cli_token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# INSTALLAZIONE E IMPORT LIBRERIE

In [ ]:
# @title
import pickle

In [ ]:
# @title
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# @title
# with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
#     top7_train_data_sim_to_each_test_data = pickle.load(f)

In [ ]:
# @title
#!pip install --upgrade pip
!pip install -q transformers --progress-bar off # Download e caricamento modelli
!pip install -q bitsandbytes --progress-bar off # Quantizzazione
!pip install -q accelerate --progress-bar off

In [ ]:
# @title
# Installazione "permanente"
!pip install -q sentencepiece --progress-bar off && touch ~/.ipython/profile_default/startup/00-first.py && echo "import <sentencepiece>" >> ~/.ipython/profile_default/startup/00-first.py

In [ ]:
# @title
!pip install -q pillow==10.0.1 --progress-bar off
!pip install -q langchain[llms] --progress-bar off

  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
# @title
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

In [ ]:
# @title
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig # Questi moduli sono dedicati al modello LLama2
from transformers import AutoModelForCausalLM, AutoTokenizer # Questi moduli permettono di scaricare qualsiasi modello e tokenizer
import pandas as pd
import torch # PyTorch
import langchain

In [ ]:
# @title
torch.cuda.empty_cache()

# DOWNLOAD E CARICAMENTO DEL MODELLO

In [ ]:
# @title
# pip install --upgrade bitsandbytes

In [ ]:
%%bash

# Models/Llama-2/llama-2-13b-chat-hf-sdg4760_MERGED/
# https://drive.google.com/file/d/1-69y3GFcXnFtSYJdc_Ww8k7uFC_EbuA_/view?usp=drive_link config.json
# https://drive.google.com/file/d/1-4igVvr8x_JTu6IrV_UISlNXyFDL7aos/view?usp=drive_link generation_config.json
# https://drive.google.com/file/d/1-AWw8xY9_Ghi_AJFztDtU1c5ofZ-H9DB/view?usp=drive_link model-00001-of-00003.safetensors
# https://drive.google.com/file/d/1-Bzqfut7DsSXv_MLs15EXi7FsgaCBIWA/view?usp=drive_link model-00002-of-00003.safetensors
# https://drive.google.com/file/d/1-KBthTYD6Z6QQJUVlHd3OwDeHYSduOTl/view?usp=drive_link model-00003-of-00003.safetensors
# https://drive.google.com/file/d/1-OzSHejMjT9d4fiYgL5Kl-cECcjT71tQ/view?usp=drive_link model.safetensors.index.json

download_data_from_google_drive () {
    html=`curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=${1}"`
    curl -Lb ./cookie "https://drive.google.com/uc?export=download&`echo ${html}|grep -Po '(confirm=[a-zA-Z0-9\-_]+)'`&id=${1}" -o ${2}
    echo "${2} downloaded"
    rm ./cookie
}

mkdir data
download_data_from_google_drive "1-69y3GFcXnFtSYJdc_Ww8k7uFC_EbuA_" "data/config.json"
download_data_from_google_drive "1-4igVvr8x_JTu6IrV_UISlNXyFDL7aos" "data/generation_config.json"
download_data_from_google_drive "1-AWw8xY9_Ghi_AJFztDtU1c5ofZ-H9DB" "data/model-00001-of-00003.safetensors"
download_data_from_google_drive "1-Bzqfut7DsSXv_MLs15EXi7FsgaCBIWA" "data/model-00002-of-00003.safetensors"
download_data_from_google_drive "1-KBthTYD6Z6QQJUVlHd3OwDeHYSduOTl" "data/model-00003-of-00003.safetensors"
download_data_from_google_drive "1-OzSHejMjT9d4fiYgL5Kl-cECcjT71tQ" "data/model.safetensors.index.json"

data/config.json downloaded
data/generation_config.json downloaded
data/model-00001-of-00003.safetensors downloaded
data/model-00002-of-00003.safetensors downloaded
data/model-00003-of-00003.safetensors downloaded
data/model.safetensors.index.json downloaded


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   661  100   661    0     0    512      0  0:00:01  0:00:01 --:--:--   512
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   183  100   183    0     0    214      0 --:--:-- --:--:-- --:--:--   214
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  % Total    % Received % Xferd  Average Speed   Tim

In [ ]:
# @title
# # @title
# model = LlamaForCausalLM.from_pretrained(  # Caricamento del modello usando la libreria dedicata. In questo caso, "ForCausal" si riferisce al modello per la generazione di testo
#     # model_id, # Nome del modello precedentemente impostato
#     'data', # Llama-7 finetuned
#     local_files_only=True,
#     load_in_4bit = True, # Quantizzazione in 4 bit (per 8 bit usare il parametro load_in_8bit)
#     torch_dtype = torch.float16 # Impostare questa codifica aiuta il modello in fase di generazione, in quanto il prompt è tipicamente float16
# )

In [ ]:
# @title
!pip install -q peft==0.4.0 --progress-bar off

In [ ]:
# @title
from peft import LoraConfig, PeftModel

In [ ]:
# model_name = model_id
# new_model = 'data'

# bnb_4bit_quant_type = "nf4"
# use_nested_quant = False
# bnb_4bit_compute_dtype = "float16"
# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# # Load the entire model on the GPU 0
# # device_map = {"": 0}

# # Reload model in FP16 and merge it with LoRA weights
# base_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map={"": "cpu"}, # vedi https://github.com/artidoro/qlora/issues/29 (hamidahmadian commented on Sep 27)
#     low_cpu_mem_usage=True,
#     return_dict=True,
#     torch_dtype=torch.float16,
#     # load_in_4bit = True, # Quantizzazione in 4 bit (per 8 bit usare il parametro load_in_8bit)
#     # bnb_4bit_quant_type=bnb_4bit_quant_type,
#     # bnb_4bit_compute_dtype=compute_dtype,
#     # bnb_4bit_use_double_quant=use_nested_quant,
# )

In [ ]:
# model = PeftModel.from_pretrained(base_model, new_model, device_map={"": "cpu"}) # vedi https://github.com/artidoro/qlora/issues/29 (hamidahmadian commented on Sep 27)
# model = model.merge_and_unload()

# model.save_pretrained("merged-model") # vedi https://github.com/artidoro/qlora/issues/29 (hamidahmadian commented on Sep 27)

# model = AutoModel.from_pretrained("data",

from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("data",
        device_map = {"": 0}, # Load the entire merged model on the GPU 0
        low_cpu_mem_usage=True,
        return_dict=True,
        torch_dtype=torch.float16,
        load_in_4bit = True # Quantizzazione in 4 bit (per 8 bit usare il parametro load_in_8bit)
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

SafetensorError: ignored

In [ ]:
model_name = model_id
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# @title
torch.cuda.empty_cache()

In [ ]:
# @title
# !nvidia-smi

In [ ]:
# @title
tokenizer = LlamaTokenizer.from_pretrained(model_id)

In [ ]:
# @title
torch.cuda.empty_cache()

In [ ]:
# @title
# model.eval()

In [ ]:
# @title
torch.cuda.empty_cache()

## TEST

In [ ]:
# @title
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=200,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
# @title
torch.cuda.empty_cache()

In [ ]:
# @title
llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
# @title
torch.cuda.empty_cache()

In [6]:
# @title
import pandas as pd
data = pd.read_excel(input_data_path)

In [ ]:
# @title
# data.shape

In [ ]:
# @title
# data.groupby('sdg').count()['text']

In [ ]:
# @title
data_sel = data

In [ ]:
# @title
# import pandas as pd
# data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(2)
# data_sel_4_few_shot_examples

In [ ]:
# @title
data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])

In [ ]:
# @title
# if USE_SAME_FEW_SHOTS==True:
#     data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])
#     data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
#         (data_sel_4_few_shot_examples.text_id==5614) |
#         (data_sel_4_few_shot_examples.text_id==2089) |
#         (data_sel_4_few_shot_examples.text_id==5665) |
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
#     ]
# else:
#     data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(7)

# data_sel_4_few_shot_examples

In [ ]:
# @title
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'

#     few_shots += input_text
#     few_shots += output_text

# print(few_shots)

In [ ]:
# @title
# template = """[INST]
# <<SYS>>
# Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The following are 7 examples of expected classification given an input text:

# """+few_shots+"""


# The text to be processed is:
# <</SYS>>

# '''
# {text}
# '''

# <<SYS>>
# Answer with label only usign the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

In [ ]:
# @title
# prompt = PromptTemplate(
#     input_variables=["text"],
#     template=template,
# )

In [ ]:
# @title
torch.cuda.empty_cache()

In [ ]:
# @title
# top7_train_data_sim_to_each_test_data

In [ ]:
# CERCA I DATI CHE SONO DA RIPROCESSARE

# import glob
# import pandas as pd

# files = glob.glob(output_data_path + "/*.pkl")
# results = []
# for file in files:
#     res = pd.read_pickle(file)
#     results.append(res)
# len(results)

# results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result']).sort_values(['cnt'])
# #results_processed

# results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
# #results_processed

# data_sel = data

# final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
# #final

# # final.to_excel(final_excel_filename)

# final = final[final.label.isna()]
# final.shape[0]

In [ ]:
# dati_da_riprocessare = list(final.text_id.values)

In [ ]:
data_sel = data.iloc[940:]
# data_sel = data[data.text_id.isin(dati_da_riprocessare)]

In [ ]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

#cnt = 8
results = []

for index, row in data_sel.iterrows():

  cnt = index
  text_id = row['text_id']
  text = row["text"].replace("{", "(").replace("}", ")").replace('\n','')
  sdg_true = row["sdg"]

  file = open(log_filename, 'a+', encoding='utf-8')

  print("\n\n##################################################")
  file.write("\n\n##################################################\n")

  print("EXPECTED_SDG:", row["sdg"], "TEXT:", text,'\n\n')
  file.write(f"EXPECTED_SDG: {row['sdg']} TEXT: " + text + "\n\n\n")

  # few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]

  # few_shots = ""

  # for train_data_id in few_shots_list:

  #   # print("##################################################")
  #   # print(train_data_id)

  #   train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

  #   # print(train_row)

  #   # print(train_row.text.values[0])
  #   # print(train_row.sdg.values[0])

  #   input_text = "Input text: \""+train_row.text.values[0].replace('\n','').replace("{", "(").replace("}", ")")+"\"\n"
  #   output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

  #   few_shots += input_text
  #   few_shots += output_text

  # print(few_shots)
  # file.write(few_shots)
  # file.write('\n')


#   template_few_shots = """[INST]
# <<SYS>>
# Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The following are """+str(number_of_shots)+""" non exhaustive examples of expected classification given an input text:

# """+few_shots+"""


# The text to be processed is:
# <</SYS>>

# '''
# {text}
# '''

# <<SYS>>
# DO NOT ABSOLUTELY INCLUDE FOR ANY REASON THE INPUT TEXT IN THE OUTPUT!

# Answer with label only usign the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

  template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".


The text to be processed is:
<</SYS>>

'''
{text}
'''

<<SYS>>
DO NOT ABSOLUTELY INCLUDE FOR ANY REASON THE INPUT TEXT IN THE OUTPUT!

Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""
  try:

    prompt = PromptTemplate(
      input_variables=["text"],
      template=template,
    )

    # print(template)

    result = llm(prompt.format(text=text))
  except Exception as e:
    print(cnt, text_id, sdg_true, e)
    result = str(e)
    output_text = "Si è verificato un errore: \n("+str(cnt)+", "+str(text_id)+", "+str(sdg_true)+", "+str(e)+")\n"
    file.write(output_text)

  result_extended = (cnt, text_id, sdg_true, result)

  print(result_extended)
  file.write(str(result_extended)+'\n')

  with open(output_data_path+f'/sdg_classification_Llama2_{llamasize}_SEMANTIC_SIMILARITY_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  results.append(result_extended)

  #cnt+=1
  file.close()
  # if cnt==5:
  #   break



In [ ]:
# @title
# data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==666]

In [ ]:
# @title
# data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==2729]

In [ ]:
# @title
# data_sel_4_few_shot_examples

In [18]:
data_sel = data

In [19]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

len(results)

1021

In [20]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed.head(3)

,cnt,text_id,sdg_true,result
0,21,3889,0,---SDG 0---\n\nAnswer removed due to exceeding...
1,22,6576,0,---SDG 3---\n\end{code}\n\nThis is an error me...
2,23,3912,0,---SDG 0---\n\n'''\nAdvertisements. FEI Eventi...


In [21]:
results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed.head(3)

,cnt,text_id,sdg_true,result,label
0,21,3889,0,---SDG 0---\n\nAnswer removed due to exceeding...,0
1,22,6576,0,---SDG 3---\n\end{code}\n\nThis is an error me...,3
2,23,3912,0,---SDG 0---\n\n'''\nAdvertisements. FEI Eventi...,0


In [22]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
final.head(3)

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,sdg_true,result,label
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,0,---SDG 0---\n,0
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,0,---SDG 0---\n\end{code}\n\nPlease let me know ...,0
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,0,---SDG 16---\n\end{code}\n\nPlease let me know...,16


In [23]:
final.to_excel(output_data_path+f"/{experiment_name}.xlsx")

In [24]:
final  = final[~final.label.isna()]

In [25]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1021
1021
              precision    recall  f1-score   support

           0       0.96      0.80      0.87        60
           1       0.77      0.72      0.74        60
           2       0.96      0.78      0.86        60
           3       0.79      0.95      0.86        60
           4       0.96      0.92      0.94        60
           5       0.92      0.92      0.92        60
           6       0.94      0.97      0.95        60
           7       0.89      0.93      0.91        60
           8       0.67      0.57      0.61        60
           9       0.71      0.90      0.79        60
          10       0.60      0.68      0.64        60
          11       0.86      0.80      0.83        60
          12       0.89      0.93      0.91        60
          13       0.93      0.87      0.90        60
          14       0.98      0.92      0.95        60
          15       0.88      0.97      0.92        61
          16       0.81      0.80      0.81        60

    accuracy    

In [26]:
classification_report_filename = output_data_path+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

'/content/gdrive/MyDrive/Colab/SDG/data/../Llama-2_FineTuning/data/output/2023.12.14-02_LLama2_7B-chat-hf_4-bit_float-16_zero-shot_FINETUNED/2023.12.14-02_LLama2_7B-chat-hf_4-bit_float-16_zero-shot_FINETUNED_classification_report.xlsx'

In [27]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_excel(classification_report_filename)

In [28]:
test = pd.read_excel(classification_report_filename)
test

,Unnamed: 0,precision,recall,f1-score,support
0,0,0.960000,0.800000,0.872727,60.000000
1,1,0.767857,0.716667,0.741379,60.000000
2,2,0.959184,0.783333,0.862385,60.000000
3,3,0.791667,0.950000,0.863636,60.000000
4,4,0.964912,0.916667,0.940171,60.000000
5,5,0.916667,0.916667,0.916667,60.000000
6,6,0.935484,0.966667,0.950820,60.000000
7,7,0.888889,0.933333,0.910569,60.000000
8,8,0.666667,0.566667,0.612613,60.000000
9,9,0.710526,0.900000,0.794118,60.000000
